# Lesson 11 - Vortex-source panel method


**Lesson09 Recap**: source-panel method to represent a circular cylinder 

**Lesson10 Recap**: Source-panel method to represent a symmetric airfoil

What if we want there to be lift? 
We need to keep in mind the **Kutta-condition** that allows us to determine wat the right amount of circulation there should be to generate lift

## A lifting-body panel method

If we calculated the flow from a free-stream and a source sheet only at an angle of nonzero, the rear stagnation point will not be located at the trailing edge. Instead, the stagnation point will be somewhere on the top surface of the airfoil. This is nont physically possible solution. 